# [ATLAS-CONF-2013-098](https://cds.cern.ch/record/1601029/files/ATLAS-CONF-2013-098.pdf)

## Single top t-channel cross-section combination @ 8 TeV

In [1]:
%matplotlib inline

In [2]:
from blue import Blue
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Load the data from a csv file and then seperate out the correlation row from the results and systematics. All uncertainties in the file are relative and so we convert them into absolute uncertainties on the measured values.

In [3]:
df = pd.read_csv('data/lhc_single_top.csv', index_col='Exp')
df.T

Exp,ATLAS,CMS,Corr
Value,95.1,80.1,0.0
StatData,2.4,7.1,0.0
StatSim,2.9,2.2,0.0
Calib,3.0,4.1,1.0
Stability,2.0,1.6,0.0
IFSR,9.1,3.1,1.0
PDF,2.8,4.6,1.0
tchgen,7.1,5.5,1.0
ttgen,3.3,0.0,0.0
hadronisation,0.8,0.0,0.0


In [4]:
corr = df.loc['Corr'][1:]
df = df.drop('Corr')
df.update((df.drop('Value', axis=1).T * df['Value'].values / 100).T)

Now we construct an instance of the `Blue` class using the Blue.iterative() class method. When we make the `Blue` class this way we apply the procedure iteratively until the change in the result is less than 1%. The iterative method scales the uncertainties based on the combined result.

In [5]:
blue = Blue.iterative(df, corr, fixed=['StatData', 'StatSim'])

We can see that the data associated to our instance of `Blue` is not the same as what we put in as the errors have been scaled.

In [6]:
np.round(blue.data.T, 2)

Exp,ATLAS,CMS
Value,95.10,80.10
StatData,2.28,5.69
StatSim,2.76,1.76
Calib,2.56,3.50
Stability,1.71,1.37
IFSR,7.76,2.64
PDF,2.39,3.92
tchgen,6.06,4.69
ttgen,2.82,0.00
hadronisation,0.68,0.00


This now behaves like a standard blue combination as if we had passed the above data as input.

In [7]:
result = blue.combined_result
result

85.283168149238861

In [8]:
uncerts = pd.Series(blue.combined_uncertainties)
uncerts

Calib            3.173899
ETMiss           0.878422
IFSR             4.413942
JER              0.966998
JES              4.423970
JVF              0.471722
LeptonEff        1.208787
LeptonRes        0.648617
LeptonScale      0.619135
Multijet         1.043018
Norm             1.644352
PDF              3.394132
PU               0.279198
Stability        1.070472
StatData         3.804595
StatSim          1.496069
Trigger          2.847816
Wjets            2.512779
btagging         4.394878
hadronisation    0.235861
tchgen           5.164439
ttgen            0.972926
dtype: float64

In [9]:
total_uncert = np.sqrt((uncerts**2).sum())
total_uncert

12.1648721560449

Equation (10) of the CONF note

In [10]:
np.round(blue.total_covariance, 0)

array([[ 269.,   84.],
       [  84.,  182.]])

The correlation is just after Eq. (10) in the CONF note:

In [11]:
print(f'Correlation = {blue.total_correlations[0, 1]:.1%}')

Correlation = 38.0%


The weights are given just before Eq. (11)

In [12]:
np.round(blue.weights, 2)

array([ 0.35,  0.65])

Eq. (11) of the CONF note is:

In [13]:
print(f'Combined cross-section = {result:.1f} +/- {total_uncert:.1f} pb')

Combined cross-section = 85.3 +/- 12.2 pb


Just after Eq. (11) the $\chi^2$ is quoted

In [14]:
blue.chi2_ndf

(0.79490106382880632, 1)

In [15]:
lumi_uncert = np.sqrt((uncerts[['Calib', 'Stability']] ** 2).sum())
lumi_uncert

3.3495591503462561

In [16]:
stat_uncert = np.sqrt((uncerts[['StatData', 'StatSim']] ** 2).sum())

In [17]:
_ = (uncerts[uncerts.index ^ {'Calib', 'Stability', 'StatData', 'StatSim'}] ** 2).sum()
syst_uncert = np.sqrt(_)

Eq. (13) is the combined result again, with the uncertainties broken down into Stat, Syst and Lumi.
Here we see a slightly different number for the lumi uncertainty, probably due to rounding somewhere?

In [18]:
print(f'Combined cross-section = {result:.1f} '
      f'+/- {stat_uncert:.1f} (stat) '
      f'+/- {syst_uncert:.1f} (syst) '
      f'+/- {lumi_uncert:.1f} (lumi) pb')

Combined cross-section = 85.3 +/- 4.1 (stat) +/- 11.0 (syst) +/- 3.3 (lumi) pb


Table 2 from the CONF note - the systematics summary table.

In [19]:
uncert_correlations = {
    'Statistics': ['StatData', 'StatSim'],
    'Luminosity': ['Calib', 'Stability'],
    'Simulation and modelling': ['IFSR', 'tchgen', 'ttgen', 'PDF', 'hadronisation'],
    'Jets': ['JER', 'JES'],
    'Backgrounds': ['Norm', 'Multijet', 'Wjets'],
    'Detector modelling': ['btagging', 'ETMiss', 'JVF', 'PU', 'LeptonEff', 'Trigger', 'LeptonRes', 'LeptonScale']
}
uncert_summary = {}
for i, j in uncert_correlations.items():
    x = np.sqrt((uncerts[j] ** 2).sum())
    uncert_summary[i] = x
uncert_summary = pd.Series(uncert_summary)
print('Source             Uncertainty (pb)')
print('-' * 35)
print(np.round(uncert_summary, 1))
print('-' * 35)
print('Total syst (excl. lumi) =  ', 
      np.round(np.sqrt((uncert_summary.drop(['Luminosity', 'Statistics']) ** 2).sum()), 1))
print('Total syst (inc. lumi) =   ', 
     np.round(np.sqrt((uncert_summary.drop('Statistics') ** 2).sum()), 1))
print('-' * 35)
print('Total uncertainty =        ',
      np.round(np.sqrt((uncert_summary ** 2).sum()), 1))

Source             Uncertainty (pb)
-----------------------------------
Backgrounds                 3.2
Detector modelling          5.5
Jets                        4.5
Luminosity                  3.3
Simulation and modelling    7.7
Statistics                  4.1
dtype: float64
-----------------------------------
Total syst (excl. lumi) =   11.0
Total syst (inc. lumi) =    11.5
-----------------------------------
Total uncertainty =         12.2


Check the stability of the combination by varying the correlation assumptions. We setup a list of dictionaries that we will use to override the defaults when we setup the iterative blue method. Then we check the difference in the result and the uncertainty with respect to the nominal result. See Table 3 of the CONF note.

In [20]:
checks = [
    {'Calib': 0.5}, {'Calib': 0}, 
    {'IFSR': 0.5, 'tchgen': 0.5, 'ttgen': 0.5, 'PDF': 0.5, 'hadronisation': 0.5},
    {'IFSR': 0, 'tchgen': 0, 'ttgen': 0, 'PDF': 0, 'hadronisation': 0},
    {'JES': 0.5}, {'JES': 1},    
    {'btagging': 0}, {'btagging': 1},
]
for i in checks:
    print('=====')
    print(i)
    tmp_blue = Blue.iterative(df, {**corr, **i}, fixed=['StatData', 'StatSim'])
    print('Shift in central value (pb) = '
          f'{tmp_blue.combined_result - result:+.1f}')
    tmp_uncert = np.sqrt((pd.Series(tmp_blue.combined_uncertainties) ** 2).sum())
    print('Shift in uncertainty (pb) = '
          f'{tmp_uncert - total_uncert:+.1f}')

=====
{'Calib': 0.5}
Shift in central value (pb) = +0.1
Shift in uncertainty (pb) = -0.1
=====
{'Calib': 0}
Shift in central value (pb) = +0.1
Shift in uncertainty (pb) = -0.2
=====
{'IFSR': 0.5, 'tchgen': 0.5, 'ttgen': 0.5, 'PDF': 0.5, 'hadronisation': 0.5}
Shift in central value (pb) = +0.4
Shift in uncertainty (pb) = -0.5
=====
{'IFSR': 0, 'tchgen': 0, 'ttgen': 0, 'PDF': 0, 'hadronisation': 0}
Shift in central value (pb) = +0.7
Shift in uncertainty (pb) = -1.1
=====
{'JES': 0.5}
Shift in central value (pb) = -0.4
Shift in uncertainty (pb) = +0.3
=====
{'JES': 1}
Shift in central value (pb) = -0.8
Shift in uncertainty (pb) = +0.6
=====
{'btagging': 0}
Shift in central value (pb) = +0.2
Shift in uncertainty (pb) = -0.2
=====
{'btagging': 1}
Shift in central value (pb) = -0.3
Shift in uncertainty (pb) = +0.2
